In [456]:
import plotly.graph_objects as go
import backtrader as bt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from IPython.display import display


# **Data**

In [457]:
ticker = "JPM"

data = yf.download(ticker, start="2018-01-01", end="2024-12-27", auto_adjust=True)

data.columns = [col[0] if col[1] == '' else col[0] for col in data.columns]


columns_to_keep = ['Open', 'High', 'Low', 'Close', 'Volume']
data = data[columns_to_keep]

data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2018-01-02,88.303798,88.623768,87.631038,88.566338,13578800
2018-01-03,88.492500,89.009375,88.180736,88.656586,11901000
2018-01-04,89.365854,90.743122,89.233897,89.926659,12953700
2018-01-05,90.108106,90.347273,88.887528,89.349365,14155000
2018-01-08,89.192675,89.629772,88.821551,89.481323,12466500
...,...,...,...,...,...
2024-12-19,232.270004,236.360001,232.270004,232.960007,11790800
2024-12-20,231.919998,239.210007,231.589996,237.600006,32348400
2024-12-23,236.070007,238.619995,234.880005,238.389999,8611500


# **Strategies**

In [458]:
class HybridATRCCIStrategy(bt.Strategy):
    params = (
        ('atr_period', 14),  # ATR periyodu
        ('cci_period', 20),  # CCI periyodu
        ('cci_threshold', 100),  # CCI eşik değeri
    )

    def __init__(self):
        self.buy_price = None
        self.sell_price = None

        self.buy_signals = []
        self.sell_signals = []

        # ATR Hesaplama
        self.atr = bt.indicators.ATR(self.data, period=self.params.atr_period)

        # CCI Hesaplama
        self.cci = bt.indicators.CommodityChannelIndex(self.data, period=self.params.cci_period)

    def next(self):
        cash = self.broker.get_cash()  # Mevcut nakit miktarı
        size = int(cash / self.data.close[0])  # İşlem büyüklüğünü hesapla

        # Alış Sinyali: CCI > eşik ve ATR artıyor
        if self.cci[0] > self.params.cci_threshold and self.atr[0] > self.atr[-1] and not self.position:
            self.buy(size=size)
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Satış Sinyali: CCI < -eşik ve ATR azalıyor
        elif self.cci[0] < -self.params.cci_threshold and self.atr[0] < self.atr[-1] and self.position:
            self.sell(size=self.position.size)
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price
                print(f"Alış Gerçekleşti: {self.buy_price:.2f}")
            elif order.issell():
                self.sell_price = order.executed.price
                print(f"Satış Gerçekleşti: {self.sell_price:.2f}")

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"İşlem Tarihi: {self.data.datetime.date(0)}")
            print(f"Alış Fiyatı: {self.buy_price:.2f}" if self.buy_price else "Alış Fiyatı: Bilinmiyor")
            print(f"Satış Fiyatı: {self.sell_price:.2f}" if self.sell_price else "Satış Fiyatı: Bilinmiyor")
            print(f"Kar/Zarar: {trade.pnl:.2f}")


In [459]:

class HybridMeanReversionVWMACDStrategy(bt.Strategy):
    params = (
        # Mean Reversion Parametreleri
        ('lookback', 50),  # Hareketli ortalama için periyot
        ('threshold', 1.5),  # Z-score eşiği

        # VWMACD Parametreleri
        ('short_period', 5),
        ('long_period', 10),
        ('signal_period', 9),
    )

    def __init__(self):

        self.buy_price = None
        self.sell_price = None

        self.buy_signals = []
        self.sell_signals = []
        
        # Mean Reversion için Hareketli Ortalama ve Standart Sapma
        self.mean = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.lookback)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=self.params.lookback)

        # VWMA Hesaplama (Kısa Dönem)
        self.vwma_short = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.short_period)
        self.vwma_long = bt.indicators.WeightedMovingAverage(self.data.close, period=self.params.long_period)

        # VMACD ve Sinyal Çizgisi
        self.vmacd = self.vwma_short - self.vwma_long
        self.signal = bt.indicators.EMA(self.vmacd, period=self.params.signal_period)

    def next(self):
        # Mean Reversion Z-score Hesaplama (Uzun Dönem)
        zscore = (self.data.close[0] - self.mean[0]) / self.std[0]

        # Alış Sinyali: Mean Reversion Alt Limitte ve VWMACD Momentum Yukarı
        if zscore < -self.params.threshold and self.vmacd[0] > self.signal[0] and not self.position:
            cash = self.broker.get_cash()  # Mevcut nakit miktarı
            size = int(cash / self.data.close[0])  # İşlem büyüklüğünü hesapla
            self.buy(size=size)
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        # Satış Sinyali: Mean Reversion Üst Limitte ve VWMACD Momentum Aşağı
        elif zscore > self.params.threshold and self.vmacd[0] < self.signal[0] and self.position:
            self.sell(size=self.position.size)  # Tüm pozisyonu kapat
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))


    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price
                print(f"Alış Gerçekleşti: {self.buy_price:.2f}")
            elif order.issell():
                self.sell_price = order.executed.price
                print(f"Satış Gerçekleşti: {self.sell_price:.2f}")

    def notify_trade(self, trade):
        if trade.isclosed:
            print(f"İşlem Tarihi: {self.data.datetime.date(0)}")
            print(f"Alış Fiyatı: {self.buy_price:.2f}" if self.buy_price else "Alış Fiyatı: Bilinmiyor")
            print(f"Satış Fiyatı: {self.sell_price:.2f}" if self.sell_price else "Satış Fiyatı: Bilinmiyor")
            print(f"Kar/Zarar: {trade.pnl:.2f}")


In [460]:
class MyStrategy(bt.Strategy):

    params = (
        ('period', 20),            # Period for SMA and Std Dev
        ('zscore_upper', 1.5),     # Z-Score upper threshold
        ('zscore_lower', -1.5),    # Z-Score lower threshold
        ('k', 2),
        ('macd_slow', 5),
        ('macd_fast', 10),
        ('macd_signal', 9), # A 9-period EMA of the MACD Line.
    )

    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []
        self.lower_bounds = []
        self.upper_bounds = []

        self.sma = bt.indicators.WeightedMovingAverage(period=self.params.period)
        self.std = bt.indicators.StandardDeviation(self.data.close, period=self.params.period)
        self.zscore = (self.data.close - self.sma) / self.std

        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal,
        )
        
        self.upper_bound = self.sma + (self.params.k * self.std)
        self.lower_bound = self.sma - (self.params.k * self.std)

    def next(self):
        self.upper_bounds.append((self.data.datetime.datetime(0), self.upper_bound[0]))
        self.lower_bounds.append((self.data.datetime.datetime(0), self.lower_bound[0]))
    
        if not self.position and self.data.close < self.lower_bound and self.macd.macd[0] > self.macd.signal[0]:
            self.buy()
            self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

        elif self.position and self.data.close > self.upper_bound and  self.macd.macd[0] < self.macd.signal[0]:
            self.sell()
            self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))


class MACD_RSI_Strategy(bt.Strategy):
    params = (
        ('macd_fast', 12),  # Fast EMA period
        ('macd_slow', 26),  # Slow EMA period
        ('macd_signal', 9),  # Signal line period
        ('rsi_period', 14),  # RSI period
        ('rsi_upper', 70),  # Overbought level
        ('rsi_lower', 30),  # Oversold level
    )

    def __init__(self):
        # Indicators
        self.macd = bt.indicators.MACD(
            self.data.close,
            period_me1=self.params.macd_fast,
            period_me2=self.params.macd_slow,
            period_signal=self.params.macd_signal,
        )
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

        # Track buy and sell signals
        self.buy_signals = []
        self.sell_signals = []

    def next(self):
        # Buy signal
        if self.macd.macd[0] > self.macd.signal[0] and self.rsi[0] < self.params.rsi_upper:
            if not self.position:
                self.buy()
                self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))
                print(f"BUY at {self.data.close[0]} on {self.data.datetime.date(0)}")

        # Sell signal
        elif self.macd.macd[0] < self.macd.signal[0] and self.rsi[0] > self.params.rsi_lower:
            if self.position:
                self.sell()
                self.sell_signals.append((self.data.datetime.datetime(0), self.data.close[0]))
                print(f"SELL at {self.data.close[0]} on {self.data.datetime.date(0)}")
                

class BuyAndHold(bt.Strategy):
    def __init__(self):
        self.buy_signals = []
        self.sell_signals = []

    def nextstart(self):
        self.order = self.buy()
        self.buy_signals.append((self.data.datetime.datetime(0), self.data.close[0]))

# **BackTrader**

In [461]:
strategies = [MyStrategy, MACD_RSI_Strategy, BuyAndHold, HybridMeanReversionVWMACDStrategy, HybridATRCCIStrategy]

strategy_dict = {}

for strategy in strategies:
    
    cerebro = bt.Cerebro()

    cerebro.addstrategy(strategy)
    # cerebro.addsizer(bt.sizers.PercentSizer, percents=99)


    data_feed = bt.feeds.PandasData(dataname=data)

    cerebro.adddata(data_feed)

    cerebro.broker.setcash(100000)

    cerebro.broker.setcommission(commission=0.001)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name="timereturn")
    cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")

    results = cerebro.run()

    strategy_dict[strategy.__name__] = {
        'cerebro': cerebro,
        'results': results
    }

BUY at 94.602783203125 on 2018-02-20
SELL at 93.45642852783203 on 2018-03-02
BUY at 97.34906005859375 on 2018-03-09
SELL at 94.80071258544922 on 2018-03-14
BUY at 91.50997924804688 on 2018-04-09
SELL at 90.16716766357422 on 2018-04-30
BUY at 91.9990234375 on 2018-05-08
SELL at 92.19795989990234 on 2018-05-24
BUY at 92.0984878540039 on 2018-06-08
SELL at 89.54549407958984 on 2018-06-14
BUY at 89.40705871582031 on 2018-07-09
SELL at 96.44931030273438 on 2018-08-10
BUY at 98.02439880371094 on 2018-09-19
SELL at 95.44086456298828 on 2018-09-27
BUY at 96.78060150146484 on 2018-10-08
SELL at 93.54956817626953 on 2018-10-10
BUY at 91.49341583251953 on 2018-10-31
SELL at 89.50443267822266 on 2018-11-23
BUY at 92.08087921142578 on 2018-11-27
SELL at 88.27916717529297 on 2018-12-06
BUY at 83.34446716308594 on 2019-01-02
SELL at 86.61865997314453 on 2019-02-07
BUY at 89.30064392089844 on 2019-02-15
SELL at 88.35307312011719 on 2019-03-01
BUY at 90.14668273925781 on 2019-03-15
SELL at 87.033226013

In [462]:
# rtot: The total return for the entire backtest period (29.66%)
# ravg: The average per-bar return (e.g., if these are daily bars, this is the average daily return)
# rnorm: The normalized (annualized) return (23.00% per year, if daily bars)
# rnorm100: The same normalized return as a percentage (23.00%)

strategy_dict['MyStrategy']['results'][0].analyzers.returns.get_analysis()

OrderedDict([('rtot', 0.001475522290608729),
             ('ravg', 8.393187091062168e-07),
             ('rnorm', 0.00021153068415544034),
             ('rnorm100', 0.021153068415544034)])

In [463]:
# 1) The current drawdown length (in bars).
# 2) The current drawdown in percentage (i.e., ~6.11%).
# 3) The current drawdown in monetary terms (e.g., $8,753.39).
# 4) The length (in bars) of the maximum drawdown observed.
# 5) The maximum drawdown percentage (~20.83%).
# 6) The maximum drawdown in monetary terms (e.g., $27,025.29).

strategy_dict['MyStrategy']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 0),
                 ('drawdown', 0.0),
                 ('moneydown', 0.0),
                 ('max',
                  AutoOrderedDict([('len', 459),
                                   ('drawdown', 0.05134898475208643),
                                   ('moneydown', 51.36248779296875)]))])

In [464]:
strategy_dict['BuyAndHold']['results'][0].analyzers.drawdown.get_analysis()

AutoOrderedDict([('len', 21),
                 ('drawdown', 0.0071384503797387614),
                 ('moneydown', 7.149993896484375),
                 ('max',
                  AutoOrderedDict([('len', 538),
                                   ('drawdown', 0.06119050412138772),
                                   ('moneydown', 61.232940673828125)]))])

In [465]:
strategy_dict['BuyAndHold']['results'][0].analyzers.timereturn.get_analysis()

OrderedDict([(datetime.datetime(2018, 1, 2, 0, 0), 0.0),
             (datetime.datetime(2018, 1, 3, 0, 0), 7.559270198687074e-07),
             (datetime.datetime(2018, 1, 4, 0, 0), 1.2700719769220825e-05),
             (datetime.datetime(2018, 1, 5, 0, 0), -5.772856276564653e-06),
             (datetime.datetime(2018, 1, 8, 0, 0), 1.3195699388734994e-06),
             (datetime.datetime(2018, 1, 9, 0, 0), 4.535710506292645e-06),
             (datetime.datetime(2018, 1, 10, 0, 0), 9.896716593793542e-06),
             (datetime.datetime(2018, 1, 11, 0, 0), 4.865456042058014e-06),
             (datetime.datetime(2018, 1, 12, 0, 0), 1.5091812259449e-05),
             (datetime.datetime(2018, 1, 16, 0, 0), -3.298807047857899e-06),
             (datetime.datetime(2018, 1, 17, 0, 0), 5.937795983124161e-06),
             (datetime.datetime(2018, 1, 18, 0, 0), 2.2266888810573704e-06),
             (datetime.datetime(2018, 1, 19, 0, 0), -2.0617443731163476e-06),
             (datetime.datetime

In [466]:
df = data.copy()
df['Date'] = df.index

run_results = strategy_dict['HybridATRCCIStrategy']['results']
strategy_instance = run_results[0]

buys = strategy_instance.buy_signals
sells = strategy_instance.sell_signals

# upper_threshold = strategy_instance.upper_bounds
# lower_threshold = strategy_instance.lower_bounds

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Close'],
    mode='lines',
    name='Price'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in buys],
    y=[signal[1] for signal in buys],
    mode='markers',
    marker=dict(size=10, color='green', symbol='triangle-up'),
    name='Buy Signal'
))

fig.add_trace(go.Scatter(
    x=[signal[0] for signal in sells],
    y=[signal[1] for signal in sells],
    mode='markers',
    marker=dict(size=10, color='red', symbol='triangle-down'),
    name='Sell Signal'
))

# fig.add_trace(go.Scatter(
#     x=[signal[0] for signal in upper_threshold],
#     y=[signal[1] for signal in upper_threshold],
#     mode='lines',
#     name='Upper Bound'
# ))

# fig.add_trace(go.Scatter(
#     x=[signal[0] for signal in lower_threshold],
#     y=[signal[1] for signal in lower_threshold],
#     mode='lines',
#     name='Lower Bound'
# ))


fig.update_layout(
    title="Buy/Sell Signals",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_white",
    height=600,
    width=1000
)

fig.show()

[(datetime.datetime(2024, 1, 20, 0, 0), 46615.93098981462), (datetime.datetime(2024, 1, 21, 0, 0), 46531.10510157404), (datetime.datetime(2024, 1, 22, 0, 0), 46505.91472830106), (datetime.datetime(2024, 1, 23, 0, 0), 46459.110305854134), (datetime.datetime(2024, 1, 24, 0, 0), 46347.01538751133), (datetime.datetime(2024, 1, 25, 0, 0), 46214.70393679485), (datetime.datetime(2024, 1, 26, 0, 0), 46105.8442234206), (datetime.datetime(2024, 1, 27, 0, 0), 46020.412813892515), (datetime.datetime(2024, 1, 28, 0, 0), 45488.21014679213), (datetime.datetime(2024, 1, 29, 0, 0), 45209.75210916528), (datetime.datetime(2024, 1, 30, 0, 0), 44682.58599899318), (datetime.datetime(2024, 1, 31, 0, 0), 43999.061583772294), (datetime.datetime(2024, 2, 1, 0, 0), 44142.343329383104), (datetime.datetime(2024, 2, 2, 0, 0), 44307.89271216214), (datetime.datetime(2024, 2, 3, 0, 0), 44471.78474309941), (datetime.datetime(2024, 2, 4, 0, 0), 44541.20607732819), (datetime.datetime(2024, 2, 5, 0, 0), 44570.63062031984)

In [467]:
# Create a Plotly figure
fig = go.Figure()

for strat_name, data_dict in strategy_dict.items():
    run_results = data_dict['results']
    
    # Usually returns is a list of strategy instances; we assume one strategy instance
    strategy_instance = run_results[0]
    
    # Extract the TimeReturn analyzer
    time_return_dict = strategy_instance.analyzers.timereturn.get_analysis()
    
    # Convert to a pandas Series
    returns_series = pd.Series(time_return_dict)
    
    # Calculate cumulative returns: (1 + r).cumprod() - 1
    cumulative_returns = (1 + returns_series).cumprod() - 1
    
    # Plot with Plotly
    fig.add_trace(go.Scatter(
        x=cumulative_returns.index,
        y=cumulative_returns.values,
        mode='lines',
        name=f"{strat_name} (Cumulative)"
    ))

# Update figure layout
fig.update_layout(
    title="Strategy Comparison: Cumulative Returns",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    template="plotly_white",
    width=900,
    height=500
)

# Show the figure
fig.show()
